In [4]:
import pandas as pd

In [ ]:
df = data = {
    'Sector': ['Retail'],
    'Tarea_Clave_Manual': ['Inventario y Cobro'],
    'Solucion_Digital_Recomendada': ['Software de Punto de Venta (POS)'],
    'Costo_Mensual_Software_MXN': [500],  
    'Riesgo_Clave': ['Error de Facturación (SAT)'],
    'Costo_Riesgo_MXN': [19700],  
    'Oportunidad_Clave': ['Aceptar Pagos Digitales'],
    'KPI_Oportunidad_Comision': [0.035],
    'KPI_Oportunidad_Aumento_Ticket': [0.20] 
}


,Sector,Tarea_Clave_Manual,Solucion_Digital_Recomendada,Costo_Mensual_Software_MXN,Riesgo_Clave,Costo_Riesgo_MXN,Oportunidad_Clave,KPI_Oportunidad_Aumento,KPI_Oportunidad_Comision
0,Retail,Inventario y Cobro,Software de Punto de Venta (POS),425.809725,Error de facturación,19700,Aceptar pagos digitales,0.2,0.035


## Seleccionar los benchmarks del módulo Retail


In [6]:
benchmarks_retail = df[df["Sector"]== "Retail"].iloc[0]
benchmarks_retail

Sector                                                    Retail
Tarea_Clave_Manual                            Inventario y Cobro
Solucion_Digital_Recomendada    Software de Punto de Venta (POS)
Costo_Mensual_Software_MXN                            425.809725
Riesgo_Clave                                Error de facturación
Costo_Riesgo_MXN                                           19700
Oportunidad_Clave                        Aceptar pagos digitales
KPI_Oportunidad_Aumento                                      0.2
KPI_Oportunidad_Comision                                   0.035
Name: 0, dtype: object

## Simular los datos que se pedirán a las pymes (bosquejo)

In [7]:
# Inputs Cálculo de eficiencia

sim_costo_horas_manual = 10  # Costo por hora de trabajo manual (inventarios/facturación)
sim_salario_admin_mes = 8000 # Salario mensual de quien haga el trabaja manual


In [8]:
# Inputs Cálculo de Riesgo

sim_probabilidad_error_sat = 0.3  # El usuario menciona que hay un 30% de probabilidad de error en SAT por captura manual (3 de 10 en riesgo)

In [9]:
# Inputs por Cálculo de Oportunidad

sim_clientes_perdidos_dia = 2  # El usuario menciona cuántos clientes pierde por día por no aceptar pagos con tarjeta
sim_ticket_promedio = 150      # Ticket promedio de venta por cliente

In [10]:
print(f"Horas manuales/semana: {sim_costo_horas_manual}")
print(f"Salario por admin/mes: {sim_salario_admin_mes}")
print(f"Probabilidad de error SAT: {sim_probabilidad_error_sat * 100}%")
print(f"Clientes perdidos/día {sim_clientes_perdidos_dia}")
print(f"Ticket Promedio: {sim_ticket_promedio}")

Horas manuales/semana: 10
Salario por admin/mes: 8000
Probabilidad de error SAT: 30.0%
Clientes perdidos/día 2
Ticket Promedio: 150


## Definir las funciones del motor para calcular el disparador de eficiencia, riesgo y oportunidad

In [21]:
# Disparador de eficiencia
# Esta función busca responder la siguiente pregunta: "¿Hacer estas tareas a mano me está costando más caro que el software que las automatiza?"

def calcular_roi_eficiencia(horas_manuales, salario_admin, costo_software):
    
    # Costo hora por trabajador
    costo_hora_hombre = salario_admin / 160  # Asumiendo 160 horas trabajadas al mes = 40 horas/semana * 4 semanas del mes

    # Costo anual de hacer el trabajo manualmente
    costo_anual_manual = (horas_manuales * 52) * costo_hora_hombre  # Muestra el costo de las horas perdidas al año realizando las tareas manuales

    # Costo anual del software
    costo_anual_software = costo_software * 12  # El costo anual del software para poder compararlo con el costo anual del trabajo manual

    # Calculo de ahorro (ROI)
    ahorro_anual = costo_anual_manual - costo_anual_software # Se calcula el ahorro anual al usar el software en lugar del trabajo manual

    return ahorro_anual, costo_anual_manual, costo_anual_software


# Disparador de riesgo
# ¿Es más barato pagar por un software que evitar multas y sanciones por errores en la captura de información fiscal?

def calcular_roi_riesgo(costo_multa_sat, costo_software_mensual, probabilidad_error):
    
    # Costo anual por multas SAT
    riesgo_anual = costo_multa_sat * probabilidad_error  # Costo de riesgo invisible al año por errores en la captura manual de información fiscal 
                                                            # (es el promedio en base a la probabilidad de error porque puede que no siempre reciba la multa de los 19 mil)
    
    # Costo anual del software
    costo_anual_software = costo_software_mensual * 12 # Costo anual del software para poder compararlo con el costo anual del riesgo invisible

    # Calculo de ahorro del riesgo
    ahorro_anual = riesgo_anual - costo_anual_software  # Se calcula el ahorro anual al usar el software en lugar de pagar multas por errores en la captura manual

    return ahorro_anual, riesgo_anual


# Disparador de oportunidad
# ¿Cuánto dinero puedo ganar si acepto pagos con tarjeta en lugar de sólo efectivo?

def calcular_roi_oportunidad(clientes_perdidos_dia, ticket_promedio, comision_tarjeta, aumento_pagos_tarjeta):    
    
    # Ganancias que se pierden por no aceptar pagos con tarjeta o podrías recuperar si aceptas pagos con tarjeta
    ganancias_ventas_recuperadas = (clientes_perdidos_dia * ticket_promedio) * 365  # Ganancias anuales que se pierden por no aceptar pagos con tarjeta

    # Ganancias brutas por aumento en pagos con tarjeta
    ganancia_total_bruta = ganancias_ventas_recuperadas * (1 + aumento_pagos_tarjeta)  # Ganancias anuales brutas al aceptar pagos con tarjeta (considerando el aumento en pagos con tarjeta)

    # Costo anual por comisiones de tarjeta
    costo_comision = ganancia_total_bruta * comision_tarjeta  # Costo anual por comisiones de tarjeta al aceptar pagos con tarjeta

    # Caclculo de la ganancia neta
    ganancia_neta_anual = ganancia_total_bruta - costo_comision  # Ganancia neta anual al aceptar pagos con tarjeta (considerando el costo por comisiones)

    return ganancia_neta_anual, ganancia_total_bruta

## Probar el motor ejecutando el diagnóstico

In [22]:
# Ejecutar Función 1 (Eficiencia)
ahorro_eficiencia, costo_manual, costo_sw = calcular_roi_eficiencia(
    sim_costo_horas_manual,
    sim_salario_admin_mes,
    benchmarks_retail['Costo_Mensual_Software_MXN']
)
print(f"Diagnóstico de Eficiencia:")
print(f"  Tu costo anual en tareas manuales es: ${costo_manual:,.2f}")
print(f"  El costo anual del software es: ${costo_sw:,.2f}")
print(f"  AHORRO ANUAL NETO: ${ahorro_eficiencia:,.2f}")
print("-" * 20)



# Ejecutar Función 2 (Riesgo)
ahorro_riesgo, riesgo_anual = calcular_roi_riesgo(
    benchmarks_retail['Costo_Riesgo_MXN'],
    benchmarks_retail['Costo_Mensual_Software_MXN'],
    sim_probabilidad_error_sat 
)
print(f"Diagnóstico de Riesgo Fiscal:")
print(f"  El costo de riesgo anualizado es de: ${riesgo_anual:,.2f}")
print(f"  El costo anual del software es: ${costo_sw:,.2f}")
if ahorro_riesgo > 0:
    print(f"Estás pagando ${ahorro_riesgo:,.2f} más por el riesgo que por el software (la solución).")
else:
    print(f"El costo del software es ${costo_sw:,.2f} mayos que tu riesgo actual.")
print("-" * 20)



# Ejecutar Función 3 (Oportunidad)

ganancia_neta, ganancia_bruta = calcular_roi_oportunidad(
    sim_clientes_perdidos_dia,
    sim_ticket_promedio,
    benchmarks_retail['KPI_Oportunidad_Comision'],
    benchmarks_retail['KPI_Oportunidad_Aumento']    
)
print(f"Diagnóistico de Oportunidad:")
print(f"  Estás perdiendo ${ganancia_bruta:,.2f} en ventas brutas por no aceptar pagos con tarjeta.")
print(f"  GANANCIA NETA ANUAL ACEPTANDO PAGOS CON TARJETA (después de comisiones): ${ganancia_neta:,.2f}")

Diagnóstico de Eficiencia:
  Tu costo anual en tareas manuales es: $26,000.00
  El costo anual del software es: $5,109.72
  AHORRO ANUAL NETO: $20,890.28
--------------------
Diagnóstico de Riesgo Fiscal:
  El costo de riesgo anualizado es de: $5,910.00
  El costo anual del software es: $5,109.72
Estás pagando $800.28 más por el riesgo que por el software (la solución).
--------------------
Diagnóistico de Oportunidad:
  Estás perdiendo $131,400.00 en ventas brutas por no aceptar pagos con tarjeta.
  GANANCIA NETA ANUAL ACEPTANDO PAGOS CON TARJETA (después de comisiones): $126,801.00
